# Fragment ion intensities Prediction 

This notebook is prepared to be run in Google [Colaboratory](https://colab.research.google.com/). In order to train the model faster, please change the runtime of Colab to use Hardware Accelerator, either GPU or TPU.

This is an extension of the original walkthrough example available [here](https://github.com/wilhelm-lab/dlomix-resources/tree/tasks/intensity/notebooks/Intensity/Example_IntensityModel_Walkthrough_colab.ipynb).

### Task 1: Learning Rate
Similar to the initial notebook, we will initialize our model and train it. The target here is to experiment with different values for the learning rate and check which learning rate would be the best to work with our task/dataset.

In [1]:
# install the mlomix package in the current environment using pip

!python -m pip install -q git+https://github.com/wilhelm-lab/dlomix.git@feature/intensity_tutorial

In [ ]:
!python -m pip install -q wandb

In [1]:
import numpy as np
import pandas as pd
import dlomix
from dlomix.models import PrositIntensityPredictor
import tensorflow as tf
from dlomix.losses import masked_spectral_distance, pearson_correlation
tf.get_logger().setLevel('ERROR')

import wandb
from wandb.keras import WandbCallback

In [ ]:
# enter project name for weights and biases
project_name = 'dlomix_intensity'

In [4]:
from dlomix.data import IntensityDataset

TRAIN_DATAPATH = 'https://raw.githubusercontent.com/wilhelm-lab/dlomix-resources/tasks/intensity/example_datasets/Intensity/proteomeTools_train_val.csv'
BATCH_SIZE = 64

int_data = IntensityDataset(data_source=TRAIN_DATAPATH,
                              seq_length=30, collision_energy_col='collision_energy', batch_size=BATCH_SIZE, val_ratio=0.2, test=False)

The code below creates a model and trains it. You should try out different values for the learning rate, which is the step size we use to update the parameters of the model (weights).  Please Refer to the initial notebook to analyze the results.

Hint: Explore the following range of values: $0.1$ to $10^{-6}$

In [ ]:
# Enter weights and biases run name. Make sure that different learning rates have different run names.
wandb.init(project=project_name, name='learning_rate')

# create model
model = PrositIntensityPredictor(seq_length=30)

optimizer = tf.keras.optimizers.SGD(learning_rate=1e-4)

# compile the model  with the optimizer and the metrics we want to use, we can add our custom time-delta metric

model.compile(optimizer=optimizer, 
            loss=masked_spectral_distance, metrics=[pearson_correlation,'mean_absolute_error', 'mse'])

history = model.fit(rtdata.train_data, validation_data=rtdata.val_data, epochs=15
                    ,callbacks=[WandbCallback(save_model=False)])


# Mark the run as finished
wandb.finish()